# Module 3 Exercise: Email Agent with HITL

Add HITL to the email agent below:
1. Between nodes
2. Within nodes

The agent is purposefully quite basic. Feel free to improve upon what has already been built!

When you've finished testing your agent in this notebook deploy your agent to LangGraph Studio.

### Email Agent (No HITL)

Creating Agent

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage
from langchain.tools import tool
from langgraph.graph import MessagesState
from langgraph.graph import StateGraph, START
from langgraph.prebuilt import tools_condition
from langgraph.checkpoint.memory import MemorySaver

# Build Model
model = ChatOpenAI(temperature=0, streaming=True, model="gpt-5-nano")

# Define system prompt
sys_msg = SystemMessage(content="You are a conversational and friendly email writing assistant." \
"You can do one of 2 things:" \
"1. Talk to the user about the email they would like to send." \
"2. Draft and send an appropriate email.")

# Create our email sending tool
# This will just return "Email Sent!" and the email info back to the model rather than actually sending an email
@tool
def send_an_email(to:str, subject:str, body:str):
    '''Send a drafted email on behalf of the user. 
    Only call this when you have received the email of the person you're sending this to.
    
    Args:
        to: str
        subject: str
        body: str'''
    
    output = f"Email Sent!\nTo: {to}\nSubject: {subject}\nBody: {body}"
    
    return output

# bind tool to model
email_model = model.bind_tools([send_an_email])

# define state
class State(MessagesState):
    pass

# create nodes
def model_node(state:State):

    return {"messages": [email_model.invoke([sys_msg] + state["messages"])]}

def tool_node(state:State):

    # last message contains param tool_calls, a list of dicts
    tool_call = state["messages"][-1].tool_calls[0]

    # make tool call
    observation = send_an_email.invoke(tool_call["args"])

    tool_message = {"role": "tool", "content" : observation, "tool_call_id": tool_call["id"]}

    return {'messages': tool_message}

# Create graph
builder = StateGraph(State)

# Add nodes
builder.add_node("model_node", model_node)
builder.add_node("tools", tool_node)

# Add edges
builder.add_edge(START, "model_node")
builder.add_conditional_edges("model_node", tools_condition)
builder.add_edge("tools", "model_node")

# Compile and view
memory = MemorySaver()
graph = builder.compile(checkpointer=memory)

# Show
graph.get_graph().print_ascii()

Running agent

In [ ]:
from langchain_core.messages import HumanMessage

# Input
input = {"messages": HumanMessage(content="Please draft an email to my boss (sean@example.com) informing them I'll be at tomorrow's meeting.")}

# Thread
thread = {"configurable": {"thread_id": "1"}}

# Run the graph
messages = graph.invoke(input, thread)
for m in messages['messages']:
    m.pretty_print()